**Name:**

**Student Number:**




In this notebook, you start by loading a trained Resnet model on CIFAR10 dataset. Then, you try to attack it with an $L_{2}$ C&W attack. Next, you work to make this model better at defending using defensive distillation, and test it against two different modes of $L_{2}$ C&W attacks again.

It is recommended to use google colab to do this homework. You can connect to your drive using the code below to use it to save and load your trained models:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load CIFAR10 data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils
import torchvision.transforms as transforms
from torchvision.models import resnet18
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

batch_size = 128

########################## Problem 1 (2  points) ###############################
# todo: Define your data loaders for training and testing                      #
################################################################################

# your code goes here

################################ End ###########################################

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Model set-up

In [ ]:
class resnet(nn.Module):
  def __init__(self, num_cls, T=1):
    super().__init__()
    self.conv = nn.Sequential(
        *list(resnet18(weights=None).children())[:-2])

    self.fc = nn.Linear(512, num_cls)
    self.temp = T

  def forward(self, x, T=None):
    if T is None:
      T = self.temp
    x = self.conv(x)
    x = torch.flatten(x, start_dim=1)
    logits = self.fc(x)
    output = torch.softmax(logits / T, dim=1)

    return logits, output

In [ ]:
# Download trained weights of the Resnet18 model on CIFAR10 dataset
!gdown 1KU4jWAwZIq0TUujAsgimLxGWUvwIEfyB

In [ ]:
# load trained Resnet18 model on CIFAR10 dataset
model = resnet(len(classes)).to(device)
model_name = "resnet18_cifar10_model_pretrained.pth"
model_PATH = "/content/" + model_name
state_dict = torch.load(model_PATH, map_location=device)
model.load_state_dict(state_dict)
model = model.to(device)

# Computing clean accuracy

In [ ]:
def standard_test(model, loader, device=device):
  correct = 0
  total = 0
  ########################## Problem 2 (4 points) ##############################
  # todo: Iterate over loader, compute the output and predicted                #
  # label, and update "correct" and "total" counters accordingly.              #
  ##############################################################################

  # your code goes here

  ################################ End #########################################
  print(f'\n Clean accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
standard_test(model, testloader)

# C&W L2 Attack on the base model

Note that for the sake of simplicity, you can ignore binary search for constant c for your implementation. However, implementing it would earn you a bonus!

In [ ]:
def cw_l2_attack(model, image, label, mode='outputs', targeted=False, c=1e-4, kappa=0, max_iter=1000, learning_rate=0.01):

    image = image.to(device)
    label = label.to(device)
    ######################### Problem 3 (8 points) #############################
    # todo: Implement L2 C&W attack in the two following modes:                #
    # 'outputs': use an f function which utilizes output probs of the model    #
    # 'logits': use an f function which utilizes logits of the model           #
    # Output of this function must be the resulting adversarial image.         #
    ############################################################################

    # your code goes here

    ################################ End #######################################

In [ ]:
def attack_test(model, attack_model, loader, mode='outputs', targeted=False, c=0.1,  device=device):
  correct = 0
  total = 0
  ########################## Problem 4 (4 points) ##############################
  # todo: Iterate over the FIRST batch of the loader                           #
  # todo: Find an adversarial example by L2 C&W attack on attack_model         #
  # todo: Compute the output and predicted label, and updated "correct" and    #
  # "total" counters accordingly.                                              #
  ##############################################################################

  # your code goes here

  ################################ End #########################################
  print(f'\n Accuracy of the network on the 64 test images of the first batch of testloader after attacking : {100 * correct // total} %')

In [ ]:
print('C&W L2 attack on the model using its outpus(probs):')
attack_test(model, model, testloader, mode='outputs', device=device)

print('\n C&W L2 attack on the model using its logits:')
attack_test(model, model, testloader, mode='logits', device=device)

# Defensive distillation

## Teacher model training

In [ ]:
T = 100
teacher = resnet(len(classes), T=T).to(device)
teacher_optim = optim.Adam(teacher.parameters())
teacher_criterion = nn.CrossEntropyLoss()

In [ ]:
# load teacher model (if you already trained it)
model_name = "teacherModel.pth"
teacher_model_PATH = "/content/drive/MyDrive/" + model_name
state_dict = torch.load(teacher_model_PATH)
teacher.load_state_dict(state_dict)
teacher = teacher.to(device)

In [ ]:
standard_train(model=teacher,
            loader=trainloader,
            num_epoch=15,
            optimizer=teacher_optim,
            criterion=teacher_criterion,
            device=device)

In [ ]:
# save teacher model (only if you just trained it)
teacher.eval()
model_name = "teacherModel.pth"
teacher_model_PATH = "/content/drive/MyDrive/" + model_name
torch.save(teacher.state_dict(), teacher_model_PATH)

## Student model training

In [ ]:
def distillation(teacher, student, loader, num_epoch, optimizer, criterion, device=device):
  ########################## Problem 5 (6 points) ##############################
  # todo: Iterate over loader in each epoch                                    #
  # todo: Compute MSE loss between student's logit and teacher's logit         #
  # todo: Take a step by the optimizer                                         #
  # todo: Monitor the training procedure                                       #
  ##############################################################################

  # your code goes here

  ################################ End #########################################

In [ ]:
T = 100
student = resnet(len(classes), T=T).to(device)
student_optim = optim.Adam(student.parameters())
std_criterion = nn.MSELoss()

In [ ]:
# load student model (only if you already trained it)
model_name = "studentModel.pth"
student_model_PATH = "/content/drive/MyDrive/" + model_name
state_dict = torch.load(student_model_PATH)
student.load_state_dict(state_dict)
student = student.to(device)

In [ ]:
distillation(teacher=teacher,
             student=student,
             loader=trainloader,
             num_epoch=15,
             optimizer=student_optim,
             criterion=std_criterion,
             device=device)

In [ ]:
# save student model (only if you just trained it)
student.eval()
model_name = "studentModel.pth"
teacher_model_PATH = "/content/drive/MyDrive/" + model_name
torch.save(student.state_dict(), teacher_model_PATH)

# C&W L2 Attack on the distilled  model

In [ ]:
print('C&W L2 attack on the student model using its outpus(probs):')
attack_test(student, student, testloader, mode='outputs', device=device)

print('\n C&W L2 attack on the student model using its logits:')
attack_test(student, student, testloader, mode='logits', device=device)

# Analyzing the results

Answer the following questions below them: (6 points)

1. Why do you think the attack success rate of the logits mode is generally better than the outputs(probs) mode?

2. Why defensive distillation completely failed in the logits mode?

3. As you have seen, defensive distillation is a mirage. Do you think that there is still some scenarios that this defense may be helpful?